# Homework 3

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

In [36]:
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import sys
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import sys
sys.path.insert(0, '../cmds') 
import statsmodels.api as sm

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per $1 face value
* Class terminology: Modified dollar duration is .1746 per $100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per $100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount $0.97 billion*
    
No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.
    

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

***

In [2]:
def price_treasury_ytm(time_to_maturity, ytm, cpn_rate,freq=2,face=100):
    c = cpn_rate/freq
    y = ytm/freq
    
    rem = freq * (time_to_maturity % (1/freq))
    tau = freq * time_to_maturity - rem
    
    if round(tau)!=tau:
        print('warning')
    else:
        tau = round(tau)    
    
    pv = 0
    for i in range(1,tau):
        pv += 1 / (1+y)**i
    
    pv = c*pv + (1+c)/(1+y)**tau
    pv *= face
    
    if rem>0:
        pv += c*face
        pv /= (1+y)**rem
        
    return pv

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [3]:
YTM = [0.04193,]
CPNRATE = [0.0450]    
SWAPRATE = [.042560]

TPRICE = 105
PAR = 100
tau0 = 30

In [4]:
price_t = price_treasury_ytm(tau0,YTM[0],CPNRATE[0])
price_t  #price of the 30-year T-bond in Nov 2008

105.21328175314193

## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

#### Clarification
List these cashflows for face value of $1B, not the $0.97B noted in the case. As mentioned in "Case Clarifications", we will not use this number. Rather, we calculate our own hedge ratio in a problem below.

In [8]:
P1 =  1e9*.04256/200
P1 #fixed leg (swap)

212800.0

In [9]:
P2 = 1e9*.0450/200
P2 #floating payment

225000.0

In [10]:
net_payment = P1-P2
net_payment

-12200.0

## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [11]:
def duration_closed_formula(tau, ytm, cpnrate=None, freq=2):

    if cpnrate is None:
        cpnrate = ytm
        
    y = ytm/freq
    c = cpnrate/freq
    T = tau * freq
        
    if cpnrate==ytm:
        duration = (1+y)/y  * (1 - 1/(1+y)**T)
        
    else:
        duration = (1+y)/y - (1+y+T*(c-y)) / (c*((1+y)**T-1)+y)

    duration /= freq
    
    return duration


In [12]:
tab_duration = pd.DataFrame(dtype=float, index=['T bond','fixed leg', 'floating leg'], columns=['duration'])
tab_duration.loc['T bond'] = duration_closed_formula(tau0, 0.0419, 0.045)
tab_duration.loc['fixed leg'] = duration_closed_formula(tau0, 0.04256)
tab_duration.loc['floating leg'] = .5
tab_duration.loc['repo'] = .5

tab_duration['dollar duration'] = tab_duration['duration'] * np.array([price_t, 100, 100, price_t * (1-0.02)])


tab_net = pd.DataFrame(dtype=float, index=['T repo','swap'], columns=['duration','dollar duration'])
tab_net.loc['T repo'] = tab_duration.loc['T bond'] - tab_duration.loc['repo']
tab_net.loc['swap'] = tab_duration.loc['fixed leg'] - tab_duration.loc['floating leg']
tab_net.loc['net'] = tab_net.loc['T repo'] - tab_net.loc['swap']

#tab_duration['dollar duration'] = tab_duration['duration'] * np.array([TPRICE, PAR, PAR, PAR])

display(tab_duration)

display(tab_net)

,duration,dollar duration
T bond,17.086964,1797.775539
fixed leg,17.212744,1721.274445
floating leg,0.500000,50.000000
repo,0.500000,51.554508


,duration,dollar duration
T repo,16.586964,1746.221031
swap,16.712744,1671.274445
net,-0.125781,74.946585


duration for the "paying-fixed" swap positive and bigger in magnitude than the T-bond

## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [13]:
hedge_ratio = tab_net.loc['swap','dollar duration'] / tab_net.loc['T repo','dollar duration']
contracts = pd.DataFrame(5e8 * np.array([hedge_ratio, -1/PAR]), index=['T repo','swap'], columns=['positions'])
contracts

,positions
T repo,4.785403e+08
swap,-5.000000e+06


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

- An increasing YTM on the T-bond indicates a lower adjusted price, resulting in a loss as we are long the T-bond.
- A declining swap rate increases the value of the fixed leg, leading to a loss since we are paying fixed.
- The floating leg of the swap offsets the repo rate, leaving no net impact.

## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$

In [14]:
prices = pd.DataFrame(index=['T bond', 'swap'],dtype=float,columns=['Nov 2008'])

prices.loc['T bond','Nov 2008'] = price_t

prices.loc['swap','Nov 2008'] = 0

prices.loc['T bond','May 2009'] = price_treasury_ytm(29.5, 0.0436, 0.045)
prices.loc['swap','May 2009'] = price_treasury_ytm(29.5, 0.0408, 0.04256) - 100

prices.style.format('${:,.2f}')

,Nov 2008,May 2009
T bond,$105.21,$102.31
swap,$0.00,$3.00


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [16]:
pnl=pd.DataFrame(dtype=float,index=['T bond','swap'],columns=['cashflow'])

pnl['cashflow'] = [[P2],[-P1]] * contracts.abs().values/1e5
pnl['capital gains'] = prices.diff(axis=1)['May 2009'].values * contracts.values[:,0]

pnl.loc['net'] = pnl.sum()

pnl['total'] = pnl.sum(axis=1)

pnl.style.format('${:,.2f}',na_rep='')


,cashflow,capital gains,total
T bond,"$1,076,715,787.00","$-1,388,667,180.27","$-311,951,393.27"
swap,"$-10,640,000.00","$-15,016,747.03","$-25,656,747.03"
net,"$1,066,075,787.00","$-1,403,683,927.30","$-337,608,140.30"


***

# 2. Factor Duration

### Data

This problem uses data from,
* `/data/yields.xlsx`
* `/data/treasury_ts_duration_2024-10-31.xlsx`

#### Load Yields

In [39]:
filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)

In [40]:
yields.head()

,1,2,5,7,10,20,30
caldt,,,,,,,
1961-06-14,2.935907,3.360687,3.623677,3.768720,3.818819,3.814210,3.815172
1961-06-15,2.932936,3.376460,3.671691,3.804225,3.862987,3.828220,3.826316
1961-06-16,2.929949,3.375670,3.685431,3.804216,3.863282,3.832922,3.830049
1961-06-19,2.920884,3.389970,3.712984,3.824557,3.886205,3.842378,3.837543
1961-06-20,2.952419,3.355796,3.685391,3.809274,3.886506,3.856465,3.845018


#### Load Prices and Durations of Two Treasuries

In [41]:
filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)
yields.head()

,1,2,5,7,10,20,30
caldt,,,,,,,
1961-06-14,2.935907,3.360687,3.623677,3.768720,3.818819,3.814210,3.815172
1961-06-15,2.932936,3.376460,3.671691,3.804225,3.862987,3.828220,3.826316
1961-06-16,2.929949,3.375670,3.685431,3.804216,3.863282,3.832922,3.830049
1961-06-19,2.920884,3.389970,3.712984,3.824557,3.886205,3.842378,3.837543
1961-06-20,2.952419,3.355796,3.685391,3.809274,3.886506,3.856465,3.845018


In [42]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [43]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME).rename(columns={207391: "price_207391", 207392: "price_207392"})
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME).rename(columns={207391: "duration_207391", 207392: "duration_207392"})

In [44]:
display(price.head())
display(duration.head())

,price_207391,price_207392
quote date,,
2019-08-09,98.882812,99.789062
2019-08-12,99.796875,102.554688
2019-08-13,99.281250,101.867188
2019-08-14,100.406250,105.179688
2019-08-15,100.882812,106.234375


,duration_207391,duration_207392
quote date,,
2019-08-09,9.289497,22.000102
2019-08-12,9.285468,22.118496
2019-08-13,9.280314,22.084308
2019-08-14,9.282750,22.228549
2019-08-15,9.282163,22.270910


### 2.1.

Construct the following yield-curve factors from the `yields` data set:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

In [45]:
maturities = list(map(int, yields.columns))
factors = pd.DataFrame(index=yields.index)
factors['level'] = yields[maturities].mean(axis=1)
factors['slope'] = yields[30] - yields[1]
factors['curvature'] = -yields[1] + 2*yields[10] - yields[30]
factors.head()

,level,slope,curvature
caldt,,,
1961-06-14,3.591027,0.879264,0.886559
1961-06-15,3.614691,0.893380,0.966721
1961-06-16,3.617360,0.900101,0.966566
1961-06-19,3.630646,0.916659,1.013983
1961-06-20,3.627267,0.892599,0.975574


### 2.2

Get the bond prices and durations for the two bonds in the data set referenced above.

#### Align the data

Align the bond pricing data with the yield factor data, so that you have data for both in the intersection of their dates.


#### Estimate the regression

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \alpha + \beta_L dx_{\text{level}} + \beta_S dx_{\text{slope}} + \beta_C dx_{\text{curvature}} + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

In [46]:
merged_df = pd.merge(price, factors, how="inner", left_index=True, right_index=True)
merged_df

,price_207391,price_207392,level,slope,curvature
2019-08-09,98.882812,99.789062,1.806375,0.416249,-0.621449
2019-08-12,99.796875,102.554688,1.715649,0.348970,-0.638499
2019-08-13,99.281250,101.867188,1.783262,0.281567,-0.669613
2019-08-14,100.406250,105.179688,1.678094,0.198027,-0.696493
2019-08-15,100.882812,106.234375,1.624473,0.217206,-0.690059
...,...,...,...,...,...
2024-05-24,86.804688,64.429688,4.642343,-0.637884,-0.838863
2024-05-28,86.484375,63.296875,4.711880,-0.536670,-0.764817
2024-05-29,86.261719,62.613281,4.768430,-0.551893,-0.787602
2024-05-30,86.550781,63.207031,4.708726,-0.548852,-0.807237


In [47]:
merged_df["dP/P_207391"] = merged_df["price_207391"].pct_change()
merged_df["dP/P_207392"] = merged_df["price_207392"].pct_change()
merged_df["d_level"] = merged_df["level"].diff()
merged_df["d_slope"] = merged_df["slope"].diff()
merged_df["d_curvature"] = merged_df["curvature"].diff()
merged_df.dropna(inplace=True)

In [48]:
Y_207391 = merged_df["dP/P_207391"]
X = merged_df[["d_level", "d_slope", "d_curvature"]]
X = sm.add_constant(X)
model = sm.OLS(Y_207391,X)
results = model.fit()
results.params

const          0.000066
d_level       -0.071967
d_slope       -0.005585
d_curvature   -0.010514
dtype: float64

In [49]:
Y_207392 = merged_df["dP/P_207392"]
X = merged_df[["d_level", "d_slope", "d_curvature"]]
X = sm.add_constant(X)
model = sm.OLS(Y_207392,X)
results = model.fit()
results.params

const          0.000092
d_level       -0.203368
d_slope       -0.130103
d_curvature    0.080332
dtype: float64

### 2.3.

Compare the "level" factor beta for each of the two treasuries with the average  duration for each bond as reported in the data set.

* How closely does the average duration for a bond compare to its "level" beta?
* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

In [50]:
avg_duration = duration.mean()
avg_duration

duration_207391     6.923664
duration_207392    19.903216
dtype: float64

From the regression, we get that the level Beta and Mathematical Duration are very close to each other

- 7.2 (Level Beta) vs 6.92 (Mathematical Duration) for 207391
- 20.3 (Level Beta) vs 19.9 (Mathematical Duration) for 207392


These values are close enough, but we need to be very precise in the case of fixed-income instruments. While regression level beta is a close approximation and can be used to roughly identify the order magnitude of sensitivity of bond prices to interest rates, for practical applications like hedging, it is better to use the mathematical form

### 2.4.

In the duration-hedged trade of `Homework 2, Section 2`, was the that trade was long or short this slope factor? 

Do you think the slope factor exposure had a large impact on the trade?

No new analysis needed, just draw a conclusion from the estimates above along with the trade construction in `HW 2, Sec 2`.

- Slope = (Yield on Long Maturity - Yield on Short Maturity). In the trade, we were going long on the short maturity and short on the long maturity. So, we are going short on the slope
- In the trade, the slope factor exposure is ~ 12.5 per (% change in yields). So the slope factor exposure is moderate.

***

# 3 Calculating Duration Via Individual Cashflows

## *Optional, not submitted*

Use the data file `../data/treasury_quotes_2024-10-31.xlsx`.

### 3.1 
Set up the cashflow matrix. 

### 3.2
Extract the Nelson-Siegel spot discount curve, as you did in `Homework 1`.

### 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)

### 3.4
How close are your duration estimates to the imputed durations given in the data source?

Report the summary statistics of the imputed durations minus your calculated durations from above.

### 3.5
Continue using your assumed discount rates of `4.5`\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

***